In [ ]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 64
compression = 1
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 29s 0us/step


In [0]:
from keras.preprocessing.image import ImageDataGenerator
DataGenerator = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
DataGenerator.fit(x_train)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43648/50000 [=========================>....] - ETA: 1:49 - loss: 1.5009 - acc: 0.4638

50000/50000 [==============================] - 927s 19ms/step - loss: 1.4490 - acc: 0.4832 - val_loss: 1.7632 - val_acc: 0.5023
Epoch 2/50
25984/50000 [==============>...............] - ETA: 6:43 - loss: 0.9602 - acc: 0.6608

50000/50000 [==============================] - 905s 18ms/step - loss: 0.8985 - acc: 0.6817 - val_loss: 1.2923 - val_acc: 0.6324
Epoch 3/50
19264/50000 [==========>...................] - ETA: 8:36 - loss: 0.7299 - acc: 0.7451

50000/50000 [==============================] - 906s 18ms/step - loss: 0.6993 - acc: 0.7555 - val_loss: 1.5992 - val_acc: 0.5660
Epoch 4/50
16768/50000 [=========>....................] - ETA: 9:17 - loss: 0.5915 - acc: 0.7920

50000/50000 [==============================] - 905s 18ms/step - loss: 0.5771 - acc: 0.7974 - val_loss: 1.1014 - val_acc: 0.7045
Epoch 5/50
15808/50000 [========>.....................] - ETA: 9:31 - loss: 0.5079 - acc: 0.8242

...

50000/50000 [==============================] - 906s 18ms/step - loss: 0.0338 - acc: 0.9884 - val_loss: 0.6200 - val_acc: 0.8879
Epoch 45/50
15104/50000 [========>.....................] - ETA: 9:44 - loss: 0.0214 - acc: 0.9921

50000/50000 [==============================] - 904s 18ms/step - loss: 0.0270 - acc: 0.9908 - val_loss: 0.7865 - val_acc: 0.8711
Epoch 46/50
15104/50000 [========>.....................] - ETA: 9:44 - loss: 0.0474 - acc: 0.9842

50000/50000 [==============================] - 904s 18ms/step - loss: 0.0339 - acc: 0.9880 - val_loss: 0.6136 - val_acc: 0.8856
Epoch 47/50
15104/50000 [========>.....................] - ETA: 9:44 - loss: 0.0253 - acc: 0.9907

50000/50000 [==============================] - 904s 18ms/step - loss: 0.0265 - acc: 0.9905 - val_loss: 0.7165 - val_acc: 0.8811
Epoch 48/50
15104/50000 [========>.....................] - ETA: 9:46 - loss: 0.0244 - acc: 0.9916

38144/50000 [=====================>........] - ETA: 3:19 - loss: 0.0242 - acc: 0.9920

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("HARDAYAL_BATCH_5_ASSIGNMENT4B_1.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('HARDAYAL_BATCH_5_ASSIGNMENT4B_1.h5')